## Modelo de Recomendación

*Se desarrolló un modelo de recomendación item-item para un sistema de recomendación en un entorno de aprendizaje automático.*\
*Se leyeron y fusionaron los datos de juegos y géneros desde archivos parquet, normalizando la fecha de lanzamiento de los juegos. Luego, se seleccionó un subconjunto de datos (en este caso, para que pudiera funcionar en la API, se tuvo que reducir la matriz a un 20%). Se calculó una matriz de similitud del coseno para determinar la similitud entre juegos basada en características como la fecha de lanzamiento y los géneros. Luego, se implementó una función 'recomendacion_juego_muestreado' que toma el ID de un juego y devuelve una lista de 5 juegos recomendados similares al juego ingresado, utilizando la matriz de similitud previamente calculada. Esta función se integra en una API mediante un método GET, donde al ingresar el ID de un juego, se obtienen las recomendaciones de juegos similares. Esto permite a los usuarios del sistema recibir recomendaciones personalizadas de juegos similares a los que les gustan.*

*Importación de las librerías necesarias*

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
df_genres = pd.read_parquet(r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\df_dummies.parquet')
df_games = pd.read_parquet(r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\Data_Clean\games_clean.parquet')

In [4]:
# Se fusionan los DataFrames df_games y df_genres en df_merged utilizando la columna 'id' 
df_merged = df_games.merge(df_genres, on='id', how='left')

# Se crea una lista llamada features que incluye la columna 'release_date' y todas las columnas de df_genres excepto la primera
features = ['release_date'] + list(df_genres.columns[1:])  

# Se crea un objeto StandardScaler para normalizar los datos de la columna 'release_date' en df_merged, escalando los valores para que tengan una media de 0 y una desviación estándar de 1.
scaler = StandardScaler()
df_merged['release_date'] = scaler.fit_transform(df_merged[['release_date']])

df_final = df_merged[['id'] + features]
df_final= df_final.merge(df_games[['id', 'app_name']], on='id', how='left')

In [8]:
# Hacer un sample de un subconjunto de ls datos (20%)
df_sampled = df_final.sample(frac=0.2, random_state=42)

# Calcular la similitud del coseno 
similarity_matrix = cosine_similarity(df_sampled[features].fillna(0))
similarity_matrix = np.nan_to_num(similarity_matrix)

In [10]:
def recomendacion_juego_muestreado(game_id, top_n=5):
    # Obtener los IDs de los juegos muestreados
    ids_juegos_muestreados = df_sampled['id'].unique()
    
    # Verificar si el ID del juego está en los juegos muestreados
    if game_id not in ids_juegos_muestreados:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    # Obtener el índice del juego en los datos muestreados
    indice_juego = df_sampled.index[df_sampled['id'] == game_id].tolist()
    
    # Verificar si se encontró el juego en los datos muestreados
    if not indice_juego:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    indice_juego = indice_juego[0]

    # Calcular los puntajes de similitud entre juegos
    puntajes_similitud = list(enumerate(similarity_matrix[indice_juego]))
    puntajes_similitud = sorted(puntajes_similitud, key=lambda x: x[1], reverse=True)

    # Obtener los índices de los juegos similares
    indices_juegos_similares = [i for i, puntaje in puntajes_similitud[1:top_n+1]]
    
    # Obtener los nombres de los juegos similares
    nombres_juegos_similares = df_sampled['app_name'].iloc[indices_juegos_similares].tolist()

    # Mensaje de recomendación
    mensaje_recomendacion = f"Juegos recomendados basados en el ID del juego {game_id} - {df_sampled['app_name'].iloc[indice_juego]}:"
    
    return [mensaje_recomendacion] + nombres_juegos_similares


In [12]:
test_game_id = "774276"
recommendations = recomendacion_juego_muestreado(test_game_id, top_n=5)

print("Recomendaciones para ID de juego", test_game_id)
for rec in recommendations:
    print(rec)

Recomendaciones para ID de juego 774276
Juegos recomendados basados en el ID del juego 774276 - Imhotep, Pyramid Builder:
TS Marketplace: Feather River Canyon Scenario Pack 01
FSX: Steam Edition - Grumman HU-16B Albatross Add-On
TANE DLC: Newcastle Shunter
FSX Steam Edition: Endelave Airport Add-On
FSX: Steam Edition - Nordborg Airfield Add-On
